In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()


if not os.environ.get("OPENAI_API_KEY"):
    print("OpenAI API key not found in environment variables")
    
if not os.environ.get("TAVILY_API_KEY"):
    print("Tavily API key not found in environment variables")

os.environ["LANGSMITH_TRACING"] = "true"

from langchain_openai import ChatOpenAI


In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results = 2)
search_results = search.invoke("What is the most technologically advanced city in the world?")
print(search_results)
tools = [search]


[{'title': '25 Most Technologically Advanced Cities in the World', 'url': 'https://finance.yahoo.com/news/25-most-technologically-advanced-cities-005705680.html', 'content': "So, without further delay, let us examine the list of the 25 most technologically advanced cities globally.\n\n25. Stockholm\n\nNumber of tech startups = 168\n\nThere are about 101,000 ICT workers in Stockholm. Stockholm contains a number of distinctive and potent elements that encourage the creation of startups. [...] Beijing has rapidly developed itself into a technological giant. The city's outstanding higher education system, reputable research foundation, and active government backing have helped it become a worldwide player in AI and sophisticated manufacturing. Beijing constructed 54,000 5G base stations in 2022 to attain basic gigabit internet access throughout the city and coverage in the 5G signal-free trade area.\n\n22. Amsterdam\n\nNumber of tech startups = 286 [...] Tokyo, the capital of Japan, plans 

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model('gpt-4o-mini', model_provider='openai')


In [5]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content = "What is the most technologically advanced city in the world in terms of IT?")])
response.content

'As of my last knowledge update in October 2023, several cities are often cited as being among the most technologically advanced in terms of information technology (IT). Notable contenders include:\n\n1. **Singapore**: Known for its Smart Nation initiative, Singapore invests heavily in IT infrastructure, smart city technologies, and data analytics. The city-state promotes a digitally connected environment with extensive use of IoT devices.\n\n2. **San Francisco/Silicon Valley**: Home to numerous tech giants and startups, the San Francisco Bay Area is a hub for innovation in IT, software development, and emerging technologies like AI and machine learning.\n\n3. **Tokyo**: Japan\'s capital is renowned for its advanced infrastructure, including high-speed internet, robotics, and technology integration in everyday life.\n\n4. **Seoul**: Recognized for its rapid technological advancement, Seoul boasts one of the fastest internet speeds worldwide and promotes initiatives in smart city develo

In [6]:
model_with_tools = model.bind_tools(tools)

print(f'ContentString: {response.content}')
print(f'ToolCalls: {response.tool_calls}')


ContentString: As of my last knowledge update in October 2023, several cities are often cited as being among the most technologically advanced in terms of information technology (IT). Notable contenders include:

1. **Singapore**: Known for its Smart Nation initiative, Singapore invests heavily in IT infrastructure, smart city technologies, and data analytics. The city-state promotes a digitally connected environment with extensive use of IoT devices.

2. **San Francisco/Silicon Valley**: Home to numerous tech giants and startups, the San Francisco Bay Area is a hub for innovation in IT, software development, and emerging technologies like AI and machine learning.

3. **Tokyo**: Japan's capital is renowned for its advanced infrastructure, including high-speed internet, robotics, and technology integration in everyday life.

4. **Seoul**: Recognized for its rapid technological advancement, Seoul boasts one of the fastest internet speeds worldwide and promotes initiatives in smart city d

In [7]:
response = model_with_tools.invoke([HumanMessage(content = "What is the most technologically advanced city in the world in terms of IT?")])

print(f'ContentString: {response.content}')
print(f'ToolCalls: {response.tool_calls}')



ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'most technologically advanced city in the world 2023 IT'}, 'id': 'call_DVtRB7srmVrbUqlnr5Umq24n', 'type': 'tool_call'}]


In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

response = agent_executor.invoke({"messages": [HumanMessage(content = "What is the most technologically advanced city in the world in terms of IT?")]})

response['messages']


[HumanMessage(content='What is the most technologically advanced city in the world in terms of IT?', additional_kwargs={}, response_metadata={}, id='3a34bf48-1c91-4b21-9ef7-f075fb9f38aa'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_95fCcuoA7BICcMCIplzfyvRj', 'function': {'arguments': '{"query":"most technologically advanced city in the world 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 94, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BJOKQ9m9EJbT6MyqW3tk2UDqrKvfH', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d230f04d-21ca-4d41-8c0b-318af5c9e1d0-0', tool_ca

In [11]:
response = agent_executor.invoke({"messages": [HumanMessage(content = "What is the most technologically advanced city in the world in terms of IT?")]})

response['messages'][-1].pretty_print()


================================== Ai Message ==================================

Based on the latest information, the most technologically advanced cities in the world in terms of IT for 2023 include:

1. **New York City, USA**: Known for its rapid adoption of smart technologies, NYC is constantly improving services through technology, making it one of the leaders in high-tech urban innovation.

2. **San Francisco, USA**: Home to Silicon Valley, San Francisco dominates the global ranking of tech cities with a deep talent pool and a culture of innovation.

3. **London, England**: Frequently recognized as one of the smartest cities globally, London is advancing its technological framework across various sectors.

For further details on these rankings and their implications, consider checking the full articles [here](https://www.savills.us/insight-and-opinion/savills-news/340224/savills-tech-cities-2023-shows-that-while-big-cities-dominate--tech-specialisms-are-accelerating-in-some-locat

In [12]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content = "What is the most technologically advanced city in the world in terms of IT?")]},
    stream_mode = "values",
):
    step['messages'][-1].pretty_print()




================================ Human Message =================================

What is the most technologically advanced city in the world in terms of IT?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_K37aqCe3qZzVqgTLp3ud8BjL)
 Call ID: call_K37aqCe3qZzVqgTLp3ud8BjL
  Args:
    query: most technologically advanced city in the world 2023 IT
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Top 10 Most Futuristic Smart Cities in the World (2023)", "url": "https://www.digi.com/blog/post/smart-cities-in-the-world", "content": "1. The Smartest City in the World: London, England [...] New York City has one of the largest populations in the U.S. and is one of the most densely populated cities on the planet. It is also constantly adopting smart technologies that improve services for its citizens. That puts it high on the list of the m

In [13]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content = "What is the most technologically advanced city in the world in terms of IT?")]},
    stream_mode = "messages",
):
    if metadata['langgraph_node'] == 'agent' and (text := step.text()):
        print(text, end = '|')



The| search| results| indicate| that| several| cities| are| considered| technologically| advanced| in| terms| of| IT|.| Not|ably|:

|1|.| **|London|,| England|**|:| Often| referred| to| as| the| smartest| city| in| the| world|,| London| incorporates| a| range| of| smart| technologies| aimed| at| improving| urban| services| for| its| citizens|.

|2|.| **|New| York| City|,| USA|**|:| Known| for| its| dense| population| and| continuous| adoption| of| smart| technologies|,| NYC| has| made| significant| upgrades| to| its| wireless| infrastructure| for| better| traffic| management| and| citizen| services|.

|For| a| comprehensive| list| and| details| on| other| cities|,| you| may| refer| to| sources| like| [|D|igi|](|https|://|www|.d|igi|.com|/blog|/post|/s|mart|-c|ities|-in|-the|-world|)| and| [|B|OSS| Magazine|](|https|://|the|boss|mag|azine|.com|/the|-most|-high|-tech|-c|ities|-in|-the|-world|/|).|

In [15]:
from langgraph.checkpoint.memory import MemorySaver

memory_saver = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer = memory_saver)

config = {"configurable": {"thread_id": "123"}}


In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content = "What is the most technologically advanced city in the world in terms of IT?")]},
    config
):
    print(chunk)
    print('----')


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HTSx1WTeqVmkr4jfcvVxaOR7', 'function': {'arguments': '{"query":"most technologically advanced city in the world 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 94, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BJOOGkYeKt0b64zj2WML38o6gzjFn', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f06ef45b-2ff1-467e-ac14-329ca2958c71-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'most technologically advanced city in the world 2023'}, 'id': 'call_HTSx1WTeqVmkr4jfcvVxaOR7', 'type': 

In [17]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content = "Hi, my name is John Doe")]},
    config
):
    print(chunk)
    print('----')


{'agent': {'messages': [AIMessage(content='Hello, John Doe! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 534, 'total_tokens': 548, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BJOOoSqg8EO6TcUkJAS6WzFHJbCYk', 'finish_reason': 'stop', 'logprobs': None}, id='run-f3871cfd-8990-424c-a8ac-4218686d9449-0', usage_metadata={'input_tokens': 534, 'output_tokens': 14, 'total_tokens': 548, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content = "What's my name?")]},
    config
):
    print(chunk)
    print('----')


{'agent': {'messages': [AIMessage(content='Your name is John Doe. How can I help you further?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 558, 'total_tokens': 573, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BJOP1d7QTtcjy4X93Z3XCaWoni3Tl', 'finish_reason': 'stop', 'logprobs': None}, id='run-8506fc1f-3fb7-4199-a29b-938fc0cab46b-0', usage_metadata={'input_tokens': 558, 'output_tokens': 15, 'total_tokens': 573, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [19]:
config = {"configurable": {"thread_id": "xyz123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content = "What's my name?")]},
    config
):
    print(chunk)
    print('----')

{'agent': {'messages': [AIMessage(content="I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. If you could tell me your name, I would be glad to use it!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 83, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BJOPMWuxFNE6fuTt2lZwF9EhejG25', 'finish_reason': 'stop', 'logprobs': None}, id='run-989d10c4-744f-4b98-b8d8-d4b88d230073-0', usage_metadata={'input_tokens': 83, 'output_tokens': 41, 'total_tokens': 124, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
